In [2]:
#https://www.youtube.com/watch?v=K_WbsFrPUCk 
#Speech recognition source
#text to speech from Stringer example

#to be able to talk into microphone
import speech_recognition as sr
#to be able to convert text and have speaker say it 
from os import system as sys
#to extract data from sites
import requests 
#to extract import html components from sites
from bs4 import BeautifulSoup
import time

#computer speak and sometimes print what is said
def say_this(text,print_):
    sys("say %s" % (text))
    if print_ != "n":
        print(text)
        
#Getting Stock information 
#Getting html from nasdaq for stocks
def get_Nasdaq_html(symbol):
    url = 'http://www.nasdaq.com/symbol/' + symbol
    response = requests.get(url)
    if response.ok:
        return response.text
    else:
        return "Error retrieving " + url
    
#Taking useless information out of stock html 
def extract_stock_data(html):
    soup = BeautifulSoup(html, "lxml")
    name = soup.select("div#qwidget_pageheader h1")[0].text
    price = soup.select("div#qwidget_lastsale")[0].text
    change = soup.select("div#qwidget_percent")[0].text
    stock= { 'Name' : name,
            "Price" : price,
            "Change" : change
    }
    return stock

def find_stock_info(symbol):
    html = get_Nasdaq_html(symbol)
    stock = extract_stock_data(html)
    print("Name: %s" % stock['Name'])
    print("Price: %s" % stock['Price'])
    print("Change: %s" % stock['Change'])

r = sr.Recognizer()

with sr.Microphone() as source:
    say_this("Say something : ", "y")
    audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        print("You said: {}".format(text))
        #several if cases to do different actions when spoken to
        
        #Find information about Destiny USA
        if "destiny" in text or "Destiny" in text:
            say_this("What would you like to know about Destiny USA?", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                
                #Several if cases to perform different Destiny information tasks
                
            except:
                print("Did not hear that")
            
            
        #Find a stock from the Nasdaq using stocksymbol
        elif "stock" in text:
            say_this("say What stock symbol should i look for? : ", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                stock_symbol = text
                find_stock_info(stock_symbol)
            except: 
                print("Could not find stock")
                
                
        #find current local time
        elif "time" in text:
            say_this("Would you like to know what time it is now? :", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                if text != "no":
                   # print("time")
                    print(time.asctime())
                else:
                    say_this("Sorry i must have misheard", "y")
            except:
                 print("Could not hear you")
    except:
        print("Could not hear you")

Say something : 
You said: what time is it
Would you like to know what time it is now? :
You said: yes m***********
Sun Dec  9 17:15:22 2018
